In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from torch.utils.data import DataLoader
from dataset import DCASE

In [7]:
root_dir = os.getcwd() + '/ADL_DCASE_DATA/development'
dataset = DCASE(root_dir, 3)

In [3]:
loader = DataLoader(dataset, batch_size=1, shuffle=True)

In [4]:
iterator = iter(loader)

data, label = iterator.next()

TypeError: Invalid file: WindowsPath('C:/Users/Tom/OneDrive - University of Bristol/.COMSM0045 Applied Deep Learning/COMSM0045/coursework/ADL_DCASE_DATA/development/audio/a026_120_150.wav')

In [ ]:
class CNN(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=self.input_shape.channels,
            out_channels=32,
            kernel_size=(5, 5),
            padding=(2, 2),
        )
        self.batch1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(
            in_channels=32,
            out_channels=64,
            kernel_size=(5, 5),
            padding=(2, 2),
        )
        self.batch2 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(
            kernel_size=(2, 2), 
            stride=(2, 2)
        )
        self.fc1 = nn.Linear(4096, 1024)
    def forward(x):
        